<div style="border:solid  green  2px; padding: 20px"> <h1 style="color: green">A/A/B- тест в приложении</h1>

# Описание проекта

Объект исследования - данные мобильного приложения по продаже продуктов питания

Цель - определить какой шрифт лучше.

Задачи: 

    - Изучить воронку продаж, проанализировать поведение пользователей
    - Исследовать результаты A/A/B-эксперимента

    Группы экспеимента: 2 контрольные со старыми шрифтами и экспериментальная — с новыми шрифтами


### Оглавление

+ Подготовка даннх
+ Изучение и проверка данных

    Сколько всего событий в логе?
    Сколько всего пользователей в логе?
    Сколько в среднем событий приходится на пользователя?
    Данными за какой период вы располагаете? 
    Много ли событий и пользователей вы потеряли, отбросив старые данные?
    Проверьте, что у вас есть пользователи из всех трёх экспериментальных групп.

+ Изучение воронки событий
    На каком шаге теряете больше всего пользователей?
    Какая доля пользователей доходит от первого события до оплаты?

+ Изучение результатов эксперимента

In [70]:
import matplotlib.pyplot as plt # выгружаем необходимые библиотеки
import pandas as pd
import numpy as np
from math import factorial
from scipy import stats as st
import math as mt
from functools import reduce
import seaborn as sb
import plotly.express as px
from plotly import graph_objects as go
import pandas as pd
import warnings
warnings.filterwarnings("ignore") 

In [71]:
pd.set_option('display.max_colwidth', -1)

In [72]:
pd.set_option('display.float_format', lambda x: '%.3f' % x) #изменение глобальной настройки отображения 

### Подготовка данных

In [73]:
app_data = pd.read_csv('/datasets/logs_exp.csv', sep="\t")

In [74]:
app_data.info()
app_data.duplicated().sum() 
app_data.dropna().reset_index(drop=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244126 entries, 0 to 244125
Data columns (total 4 columns):
EventName         244126 non-null object
DeviceIDHash      244126 non-null int64
EventTimestamp    244126 non-null int64
ExpId             244126 non-null int64
dtypes: int64(3), object(1)
memory usage: 7.5+ MB


,EventName,DeviceIDHash,EventTimestamp,ExpId
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248
...,...,...,...,...
244121,MainScreenAppear,4599628364049201812,1565212345,247
244122,MainScreenAppear,5849806612437486590,1565212439,246
244123,MainScreenAppear,5746969938801999050,1565212483,246
244124,MainScreenAppear,5746969938801999050,1565212498,246


In [75]:
app_data.set_axis(['event_name', 'uid', 'event_timestamp', 'expld'], axis = 'columns',\
                inplace = True)
app_data.head(15)

,event_name,uid,event_timestamp,expld
0,MainScreenAppear,4575588528974610257,1564029816,246
1,MainScreenAppear,7416695313311560658,1564053102,246
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248
3,CartScreenAppear,3518123091307005509,1564054127,248
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248
5,CartScreenAppear,6217807653094995999,1564055323,248
6,OffersScreenAppear,8351860793733343758,1564066242,246
7,MainScreenAppear,5682100281902512875,1564085677,246
8,MainScreenAppear,1850981295691852772,1564086702,247
9,MainScreenAppear,5407636962369102641,1564112112,246


In [76]:
display(app_data.isnull().sum())
app_data['date_time'] = pd.to_datetime(app_data['event_timestamp'], unit='s') #выделим отдельный столбец с 
#удобным форматом


event_name         0
uid                0
event_timestamp    0
expld              0
dtype: int64

In [77]:
app_data['event_date'] = app_data['date_time'].astype('datetime64[D]')

In [78]:
def group_name(expld):
    '''Возвращает название группы'''
    if expld == 246:
        return "A-1"
    if expld == 247:
        return "A-2"
    if expld == 248:
        return "B"

app_data['group'] = app_data['expld'].apply(group_name)
app_data.head(15)

,event_name,uid,event_timestamp,expld,date_time,event_date,group
0,MainScreenAppear,4575588528974610257,1564029816,246,2019-07-25 04:43:36,2019-07-25,A-1
1,MainScreenAppear,7416695313311560658,1564053102,246,2019-07-25 11:11:42,2019-07-25,A-1
2,PaymentScreenSuccessful,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25,B
3,CartScreenAppear,3518123091307005509,1564054127,248,2019-07-25 11:28:47,2019-07-25,B
4,PaymentScreenSuccessful,6217807653094995999,1564055322,248,2019-07-25 11:48:42,2019-07-25,B
5,CartScreenAppear,6217807653094995999,1564055323,248,2019-07-25 11:48:43,2019-07-25,B
6,OffersScreenAppear,8351860793733343758,1564066242,246,2019-07-25 14:50:42,2019-07-25,A-1
7,MainScreenAppear,5682100281902512875,1564085677,246,2019-07-25 20:14:37,2019-07-25,A-1
8,MainScreenAppear,1850981295691852772,1564086702,247,2019-07-25 20:31:42,2019-07-25,A-2
9,MainScreenAppear,5407636962369102641,1564112112,246,2019-07-26 03:35:12,2019-07-26,A-1


+ Отредактированы названия столбцов, приведены к нижнему регистру
+ Пропусков нет
+ Изменен тип данных в столбце с датой и временем, выделен отдельный столбец с датами
+ Добавлен столбец с расшифровкой по эксперементируемым группам



Данные изучены и подготовлены к дальнейшему анализу.

## Изучение и проверка данных

In [79]:
print('Всего уникальных событий в логе:', app_data['event_name'].nunique())
print('Всего событий в логе:', app_data['event_name'].count())
print('Всего уникальных пользователей в логе:', app_data['uid'].nunique())

Всего уникальных событий в логе: 5
Всего событий в логе: 244126
Всего уникальных пользователей в логе: 7551


In [80]:
event_id = app_data.pivot_table(index='uid', values='event_name', aggfunc = 'count')
print('В среднем на пользователя приходится событий:', event_id['event_name'].mean())

В среднем на пользователя приходится событий: 32.33028737915508


### Определение периода данных для анализа

In [81]:
print('Минимальная дата :', app_data['date_time'].min())
print('Максимальная дата :', app_data['date_time'].max())

Минимальная дата : 2019-07-25 04:43:36
Максимальная дата : 2019-08-07 21:15:17


Данные предоставлены за 13 дней

In [82]:
date_data = app_data.pivot_table(index='event_date', columns='event_name', aggfunc='count').reset_index()
date_data

event_date        date_time                                      \
event_name            CartScreenAppear MainScreenAppear OffersScreenAppear   
0          2019-07-25 2.000            4.000            1.000                
1          2019-07-26 6.000            16.000           4.000                
2          2019-07-27 4.000            39.000           9.000                
3          2019-07-28 24.000           53.000           16.000               
4          2019-07-29 23.000           132.000          20.000               
5          2019-07-30 49.000           280.000          51.000               
6          2019-07-31 258.000          1250.000         374.000              
7          2019-08-01 6231.000         17957.000        6991.000             
8          2019-08-02 6529.000         16812.000        6926.000             
9          2019-08-03 6381.000         14845.000        6797.000             
10         2019-08-04 6162.000         15216.000        6490.000             
11         2019-08-05 6257.000         18020.000        6672.000             
12         2019-08-06 5744.000         18771.000        6533.000             
13         2019-08-07 5061.000         15810.000        5941.000             

                                             event_timestamp                   \
event_name PaymentScreenSuccessful Tutorial CartScreenAppear MainScreenAppear   
0          2.000                   nan      2.000            4.000              
1          3.000                   2.000    6.000            16.000             
2          1.000                   2.000    4.000            39.000             
3          12.000                  nan      24.000           53.000             
4          9.000                   nan      23.000           132.000            
5          33.000                  nan      49.000           280.000            
6          140.000                 9.000    258.000          1250.000           
7          4802.000                248.000  6231.000         17957.000          
8          5171.000                168.000  6529.000         16812.000          
9          5166.000                162.000  6381.000         14845.000          
10         5025.000                140.000  6162.000         15216.000          
11         5043.000                121.000  6257.000         18020.000          
12         4644.000                130.000  5744.000         18771.000          
13         4262.000                70.000   5061.000         15810.000          

                                                       ...            group  \
event_name OffersScreenAppear PaymentScreenSuccessful  ... CartScreenAppear   
0          1.000              2.000                    ... 2.000              
1          4.000              3.000                    ... 6.000              
2          9.000              1.000                    ... 4.000              
3          16.000             12.000                   ... 24.000             
4          20.000             9.000                    ... 23.000             
5          51.000             33.000                   ... 49.000             
6          374.000            140.000                  ... 258.000            
7          6991.000           4802.000                 ... 6231.000           
8          6926.000           5171.000                 ... 6529.000           
9          6797.000           5166.000                 ... 6381.000           
10         6490.000           5025.000                 ... 6162.000           
11         6672.000           5043.000                 ... 6257.000           
12         6533.000           4644.000                 ... 5744.000           
13         5941.000           4262.000                 ... 5061.000           

                                                                        \
event_name MainScreenAppear OffersScreenAppear PaymentScreenSuccessful   
0          4.000            1.000              2.000

Данные предоставлены неравномерно. Полные данные представлены только с 31.08.2019 Для дальнейшего анализа оставим данные за последнюю неделю: с 01.08.2019 по 07.08.2019.

In [83]:
new_app_data = app_data.query('event_date >= "2019-08-01"') #датасет с одинаковой полностью данных
new_app_data.head()

,event_name,uid,event_timestamp,expld,date_time,event_date,group
2828,Tutorial,3737462046622621720,1564618048,246,2019-08-01 00:07:28,2019-08-01,A-1
2829,MainScreenAppear,3737462046622621720,1564618080,246,2019-08-01 00:08:00,2019-08-01,A-1
2830,MainScreenAppear,3737462046622621720,1564618135,246,2019-08-01 00:08:55,2019-08-01,A-1
2831,OffersScreenAppear,3737462046622621720,1564618138,246,2019-08-01 00:08:58,2019-08-01,A-1
2832,MainScreenAppear,1433840883824088890,1564618139,247,2019-08-01 00:08:59,2019-08-01,A-2


In [84]:
print('Всего уникальных событий в отредактированных данных:', new_app_data['event_name'].nunique())
print('Всего событий в отредактированных данных:', new_app_data['event_name'].count())
print('Всего уникальных пользователей в отредактированных данных:', new_app_data['uid'].nunique())
print('Потеряно данных о пользователях:', app_data['uid'].nunique() - new_app_data['uid'].nunique())
print('Потеряно данных о событиях:', app_data['event_name'].count() - new_app_data['event_name'].count())
print('Потеряно данных о событиях в %:', (app_data['event_name'].count() - new_app_data['event_name'].count()) /\
     len(app_data['event_name']))


Всего уникальных событий в отредактированных данных: 5
Всего событий в отредактированных данных: 241298
Всего уникальных пользователей в отредактированных данных: 7534
Потеряно данных о пользователях: 17
Потеряно данных о событиях: 2828
Потеряно данных о событиях в %: 0.011584181938834865


In [85]:
expld_id = new_app_data.pivot_table(index='group', values='uid', aggfunc = 'count')
expld_id #данные представлены всеми группами эксперимента

,uid
group,
A-1,79425
A-2,77147
B,84726


Данные отредактированны по принципу полного объема представленных событий. При редактировании потеряно 0,003% от обзего объема данных. Отредактированные данные представлены всеми группами эксперимента.

## Воронка событий

In [86]:
new_app_data['event_name'].value_counts().sort_values()

Tutorial                   1039  
PaymentScreenSuccessful    34113 
CartScreenAppear           42365 
OffersScreenAppear         46350 
MainScreenAppear           117431
Name: event_name, dtype: int64

Чаще всего встречается событие MainScreenAppear - посещение главной страницы.

###  Сколько пользователей совершали каждое из событий

In [87]:
print('Количество и доля пользователей, совершивших каждое событие')
uid_count = new_app_data.groupby(['event_name','group']).agg({'uid':'nunique'}).\
sort_values(by='uid',ascending=False).reset_index()
uid_count['percent'] = uid_count['uid'] / 7286 * 100
print(uid_count)

Количество и доля пользователей, совершивших каждое событие
                 event_name group   uid  percent
0   MainScreenAppear         B     2493 34.216  
1   MainScreenAppear         A-2   2476 33.983  
2   MainScreenAppear         A-1   2450 33.626  
3   OffersScreenAppear       A-1   1542 21.164  
4   OffersScreenAppear       B     1531 21.013  
5   OffersScreenAppear       A-2   1520 20.862  
6   CartScreenAppear         A-1   1266 17.376  
7   CartScreenAppear         A-2   1238 16.991  
8   CartScreenAppear         B     1230 16.882  
9   PaymentScreenSuccessful  A-1   1200 16.470  
10  PaymentScreenSuccessful  B     1181 16.209  
11  PaymentScreenSuccessful  A-2   1158 15.893  
12  Tutorial                 A-2   283  3.884   
13  Tutorial                 B     279  3.829   
14  Tutorial                 A-1   278  3.816   


Предположительно, последовательность событий может быть следующей: MainScreenAppear - Tutorial -  OffersScreenAppear - CartScreenAppear - PaymentScreenSuccessful. Тем не менее, событие Tutorial (руководство) совершает только 9% всех юзеров. Данное событие логично не учитывать при расчете воронки. Также, стоит помнить, что пользователь может совершить первое действие, пропустив возможные (например по прямой ссылке).

### Воронка событий

In [88]:
uid_count1 = new_app_data.pivot_table(index='uid', columns='event_name', values='date_time', aggfunc='min')
uid_count1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7534 entries, 6888746892508752 to 9222603179720523844
Data columns (total 5 columns):
CartScreenAppear           3734 non-null datetime64[ns]
MainScreenAppear           7419 non-null datetime64[ns]
OffersScreenAppear         4593 non-null datetime64[ns]
PaymentScreenSuccessful    3539 non-null datetime64[ns]
Tutorial                   840 non-null datetime64[ns]
dtypes: datetime64[ns](5)
memory usage: 353.2 KB


In [89]:
print('Доля посетивших главную страницу из общего количества пользователей:', uid_count1['MainScreenAppear'].\
      count() / len(uid_count1)*100)
print('Конверсия из посещения главной страницы в просмотр оффера:', uid_count1['OffersScreenAppear'].count() /\
     uid_count1['MainScreenAppear'].count()*100)
print('Конверсия из просмотра оффера в переход в корзину:', uid_count1['CartScreenAppear'].count() /\
     uid_count1['OffersScreenAppear'].count()*100)
print('Конверсия из перехода в корзину в успешную оплату:', uid_count1['PaymentScreenSuccessful'].count() /\
     uid_count1['CartScreenAppear'].count()*100)
print('Доля пользователей, дошедших до успешной оплаты от общего количества:',\
      uid_count1['PaymentScreenSuccessful'].count() / len(uid_count1)*100)

Доля посетивших главную страницу из общего количества пользователей: 98.47358640828246
Конверсия из посещения главной страницы в просмотр оффера: 61.908613020622724
Конверсия из просмотра оффера в переход в корзину: 81.29762682342695
Конверсия из перехода в корзину в успешную оплату: 94.7777182645956
Доля пользователей, дошедших до успешной оплаты от общего количества: 46.973719139899124


In [90]:
fig = go.Figure()
 
for name in ['A-1','A-2','B']:
    funnel = uid_count.query('group ==@name')
   
 
   
    fig.add_trace(
       go.Funnel(
         name = name,
          y = funnel['event_name'],
          x = funnel['uid'],
           orientation = "h",
        textinfo = "value+percent initial"
       )
    )
fig.update_layout(title='Воронка событий')
fig.show()

Больше всего пользователей "теряется" на этапе перехода с главной страницы на страницу с выбором. Вероятно, проблема в формированием потребности. Возможно, 1,5% пользователей заходят в приложение из пуш уведомлений ошибочно или заходят в приложение для уточнения информации, не касающейся заказа (контактная информация, отследить заказ, оформленный ранее). 

Также, 19% на заканчивает оформление заказа и закрывает приложение, не заходя в корзину. 

Только 47% пользователей завершает использование приложения оплатой.

## Результаты эксперимента

In [91]:
group_data = new_app_data.pivot_table\
(index='event_name', columns='group', values='uid', aggfunc='nunique').sort_values\
(by = 'A-1', ascending=False)
group_data['AA'] = group_data['A-1'] + group_data['A-2']
group_data

group,A-1,A-2,B,AA
event_name,,,,
MainScreenAppear,2450,2476,2493,4926
OffersScreenAppear,1542,1520,1531,3062
CartScreenAppear,1266,1238,1230,2504
PaymentScreenSuccessful,1200,1158,1181,2358
Tutorial,278,283,279,561


 ### Будет ли отличие между группами статистически достоверным? Проверка гипотез

In [92]:
uid_count

,event_name,group,uid,percent
0,MainScreenAppear,B,2493,34.216
1,MainScreenAppear,A-2,2476,33.983
2,MainScreenAppear,A-1,2450,33.626
3,OffersScreenAppear,A-1,1542,21.164
4,OffersScreenAppear,B,1531,21.013
5,OffersScreenAppear,A-2,1520,20.862
6,CartScreenAppear,A-1,1266,17.376
7,CartScreenAppear,A-2,1238,16.991
8,CartScreenAppear,B,1230,16.882
9,PaymentScreenSuccessful,A-1,1200,16.470


+ Количество пользователей в различных группах различается не более, чем на 1%
+ Для всех групп фиксируют и отправляют в системы аналитики данные об одном и том же 

In [93]:
"Применим гипотезу о равенстве долей"
#Нулевая гипотеза: Доли по количеству пользователей тестируемых групп равны.
#Альтернативная гипотеза: Доли по количеству пользователей тестируемых групп отличаются.
def stat_realibility (df, column1, column2, alpha):
   
    events = df.index 
    for i in range(1, 5):
        alpha = alpha
        bonferroni = alpha / 16 # Чтобы снизить групповую вероятность ошибки первого рода и 
        #скорректировать требуемые уровни значимости применим метод Бонферрони
        row1 = np.array([df.loc[events[i-1], column1], df.loc[events[i-1], column2]])
        row2 = np.array([df.loc[events[i], column1], df.loc[events[i], column2]])
       
        p1 = row2[0] / row1[0]
        p2 = row2[1] / row1[1]
        p_combined = (row2[0] + row2[1]) / (row1[0] + row1[1])
        difference = p1 - p2
        z_value = difference / mt.sqrt(p_combined * (1 - p_combined) * (1/row1[0] + 1/row1[1]))
        distr = st.norm(0, 1)
        p_value = (1 - distr.cdf(abs(z_value))) * 2
        print('p-значение: ', p_value)
        if (p_value < bonferroni):
            print("Отвергаем нулевую гипотезу: между долями есть значимая разница")
        else:
            print("Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными")


In [94]:
group_data

group,A-1,A-2,B,AA
event_name,,,,
MainScreenAppear,2450,2476,2493,4926
OffersScreenAppear,1542,1520,1531,3062
CartScreenAppear,1266,1238,1230,2504
PaymentScreenSuccessful,1200,1158,1181,2358
Tutorial,278,283,279,561


In [95]:
y = stat_realibility (group_data, "A-1","A-2", 0.05)
print(y)

p-значение:  0.2622344959255778
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.6393754713145263
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.18243439408067652
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.46833167209360305
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
None


In [96]:
x = stat_realibility (group_data, "A-1","B", 0.05)
print(x)

p-значение:  0.2684281587249231
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.2112170818054686
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.14295341270419004
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.7920938967629971
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
None


In [97]:
z = stat_realibility (group_data, "A-2","B", 0.05)
print(z)

p-значение:  0.9869365252844515
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.4364903513062255
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.005677465395736325
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.6447626122251844
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
None


In [98]:
a = stat_realibility (group_data, "AA","B", 0.05)
print(a)

p-значение:  0.5308589432077624
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.23905954279947372
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.017114434319547067
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.9121675205768531
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
None


По итогу множественного теста, можно сделать вывод о корректности разделения на группы.

Изменим уровень статистической значимости, чтобы минимизировать возможность ложного результата.

In [99]:
y1 = stat_realibility (group_data, "A-1","A-2", 0.1)
print(y1)

p-значение:  0.2622344959255778
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.6393754713145263
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.18243439408067652
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.46833167209360305
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
None


In [100]:
x1 = stat_realibility (group_data, "A-1","B", 0.1)
print(x1)

p-значение:  0.2684281587249231
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.2112170818054686
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.14295341270419004
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.7920938967629971
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
None


In [101]:
z1 = stat_realibility (group_data, "A-2","B", 0.1)
print(z1)

p-значение:  0.9869365252844515
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.4364903513062255
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.005677465395736325
Отвергаем нулевую гипотезу: между долями есть значимая разница
p-значение:  0.6447626122251844
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
None


In [102]:
a1 = stat_realibility (group_data, "AA","B", 0.1)
print(a1)

p-значение:  0.5308589432077624
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.23905954279947372
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.017114434319547067
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
p-значение:  0.9121675205768531
Не получилось отвергнуть нулевую гипотезу, нет оснований считать доли разными
None


Уменьшение уровня статистической разницы не изменило результат. Нет оснований считать доли групп разными по всем действиям во всех группах.

## Вывод

По итогам анализа воронки продаж, можно сделать выводы:
+ Больше всего пользователей "теряется" на этапе перехода с главной страницы на страницу с выбором. Вероятно, проблема в формированием потребности; 
+ 19% на заканчивает оформление заказа и закрывает приложение, не заходя в корзину;
+ 47% пользователей завершает использование приложения оплатой.

По итогам анализа A/A/B-эксперимента, можно сделать вывод:
+ На каждом этапе в каждой группе разницы в долях не обнаружено.

Изменение шрифта не принесло статистически значимых именений в поведении пользователей ни в большую, ни в меньшую сторону. Новый шрифт не привел к росту конверсии в продажу и не "смутил" пользователей. 
Отталкиваясь от задачи эксперимента, можно считать тест оконченым успешно. Новые шрифты не изменили результаты по продажам.